# Projeto Final de Curso

## Importando Dados

### Ibovespa

In [3]:
import yfinance as yf

symbol = '^BVSP'  # O símbolo do Ibovespa no Yahoo Finance
start_date = '2000-01-01'
end_date = '2024-11-08'

df_ibovespa = yf.download(symbol, start=start_date, end=end_date)
df_ibovespa.reset_index(inplace=True)
df_ibovespa.to_csv("data/ibovespa_2000_2024.csv", index=False)


[*********************100%%**********************]  1 of 1 completed


### Selic

In [4]:
import pandas as pd

taxas_selic = [17.56, 14.13, 11.25, 11.82, 9.50, 9.37,	11.04,	 # dados gov.br
        8.17,	7.92,	10.40,	12.54,	13.20,	
        9.53,	6.24,	5.79,	2.72,	
        4.35,	11.73,	12.31,	10.38]

anos = [2005, 2006, 2007, 2008, 2009,
        2010, 2011, 2012, 2013, 2014,
        2015, 2016, 2017, 2018, 2019,
        2020, 2021, 2022, 2023, 2024]



taxas_selic = [round(t / 100, 4) for t in taxas_selic]


# Criando o DataFrame
df = pd.DataFrame({"Ano": anos, "Taxa Selic Anual Média": taxas_selic})

# Salvando os dados em um arquivo CSV
df.to_csv('taxa_selic_anual_2005_2024.csv', index=False)
print("Dados salvos no arquivo 'taxa_selic_anual_2005_2024.csv'.")


Dados salvos no arquivo 'taxa_selic_anual_2005_2024.csv'.


## Modelos que utilizam a função recompensa "padrão"

### Dense

In [ ]:
#Importar 
import os
import pandas as pd
import logging
import coloredlogs
from docopt import docopt
from trading_bot.agent_Dense import Agent
from trading_bot.methods_default import train_model, evaluate_model
from trading_bot.utils import (
    get_stock_data,
    get_data_with_date,
    format_currency,
    format_position,
    show_train_result,
    switch_k_backend_device
)

#Dados
dados = "data/ibovespa_2000_2024.csv"

#Import como lista
dados_lista = get_stock_data(dados)
dados_lista_data = get_data_with_date(dados)

#Valores janela movel
janela_inicial = 1260  
delta_janela = 1260
final_dados = len(dados_lista)
cont = 0
inicio_janela = 0   
final_janela = janela_inicial   
 
#Eps treino
ep_count=int(3)

while cont < final_dados:
    print(inicio_janela, final_janela)
    dados_treino = dados_lista[inicio_janela:final_janela]
    agent = Agent(int(10), strategy='dqn', pretrained=False, model_name=None)
    ep_count=int(3)
    episode = None
    for episode in range(1, ep_count + 1):
        treino = train_model(agent, episode, dados_treino, ep_count=ep_count,
                                   batch_size=int(32), ws=int(10))

    # Se tem dados para o próximo teste   
    if final_janela + delta_janela + 10 < final_dados:
        inicio_dados_teste = final_janela
        fim_dados_teste = inicio_dados_teste + delta_janela + 1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        inicio_janela += delta_janela
        final_janela += delta_janela

    else:
        inicio_dados_teste = final_janela
        fim_dados_teste = final_dados+1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        cont = final_dados
    
    if len(dados_teste) > 10: 
        #Teste
        initial_offset = dados_teste[1] - dados_teste[0]
        agent = Agent(int(10), strategy='dqn', pretrained=True, model_name='dqn.h5')
        profit, _ = evaluate_model(agent, dados_teste, dados_teste_data, ws=int(10), debug=True)

        # Salvar resultados teste
        df_negociacoes_valores = pd.read_csv('negociações_valores.csv')
        df_profit_by_date = pd.read_csv('profit_by_date.csv')
        df_operacoes = pd.read_csv('operação_dia.csv')

        df_negociacoes_valores_historico = pd.read_csv('negociações_valores_historico_5ano_Dense_default.csv') 
        df_negociacoes_valores_historico = pd.concat([df_negociacoes_valores_historico, df_negociacoes_valores], ignore_index=True)

        df_profit_by_date_historico = pd.read_csv('profit_by_date_historico_5ano_Dense_default.csv')  
        df_profit_by_date_historico = pd.concat([df_profit_by_date_historico, df_profit_by_date], ignore_index=True)

        df_operacoes_historico = pd.read_csv('operacoes_dia_historico_5ano_Dense_default.csv')  
        df_operacoes_historico = pd.concat([df_operacoes_historico, df_operacoes], ignore_index=True)

        df_negociacoes_valores_historico.to_csv('negociações_valores_historico_5ano_Dense_default.csv', index=False)
        df_profit_by_date_historico.to_csv('profit_by_date_historico_5ano_Dense_default.csv', index=False)
        df_operacoes_historico.to_csv('operacoes_dia_historico_5ano_Dense_default.csv', index=False)

        arquivo = "models/dqn.h5"
        inicio = str(inicio_dados_teste)
        fim = str(fim_dados_teste)
        nome_novo = "models/testados_janelas_moveis/dqn_"+inicio+"_"+fim +"_Dense_default.h5"

        if os.path.exists(arquivo):
            # Renomeia o arquivo
            os.rename(arquivo, nome_novo)

### LSTM

In [ ]:
#Importar 
import os
import pandas as pd
import logging
import coloredlogs
from docopt import docopt
from trading_bot.agent_LSTM import Agent
from trading_bot.methods_default import train_model, evaluate_model
from trading_bot.utils import (
    get_stock_data,
    get_data_with_date,
    format_currency,
    format_position,
    show_train_result,
    switch_k_backend_device
)

#Dados
dados = "data/ibovespa_2000_2024.csv"

#Import como lista
dados_lista = get_stock_data(dados)
dados_lista_data = get_data_with_date(dados)

#Valores janela movel
janela_inicial = 1260  
delta_janela = 1260
final_dados = len(dados_lista)
cont = 0
inicio_janela = 0  
final_janela = janela_inicial   
 
#Eps treino
ep_count=int(3)

while cont < final_dados:
    print(inicio_janela, final_janela)
    dados_treino = dados_lista[inicio_janela:final_janela]
    agent = Agent(int(10), strategy='dqn', pretrained=False, model_name=None)
    ep_count=int(3)
    episode = None
    for episode in range(1, ep_count + 1):
        treino = train_model(agent, episode, dados_treino, ep_count=ep_count,
                                   batch_size=int(32), ws=int(10))

    # Se tem dados para o próximo teste   
    if final_janela + delta_janela + 10 < final_dados:
        inicio_dados_teste = final_janela
        fim_dados_teste = inicio_dados_teste + delta_janela + 1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        inicio_janela += delta_janela
        final_janela += delta_janela

    else:
        inicio_dados_teste = final_janela
        fim_dados_teste = final_dados+1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        cont = final_dados
    
    if len(dados_teste) > 10: 
        #Teste
        initial_offset = dados_teste[1] - dados_teste[0]
        agent = Agent(int(10), strategy='dqn', pretrained=True, model_name='dqn.h5')
        profit, _ = evaluate_model(agent, dados_teste, dados_teste_data, ws=int(10), debug=True)

        # Salvar resultados teste
        df_negociacoes_valores = pd.read_csv('negociações_valores.csv')
        df_profit_by_date = pd.read_csv('profit_by_date.csv')
        df_operacoes = pd.read_csv('operação_dia.csv')

        df_negociacoes_valores_historico = pd.read_csv('negociações_valores_historico_5ano_LSTM_default.csv') 
        df_negociacoes_valores_historico = pd.concat([df_negociacoes_valores_historico, df_negociacoes_valores], ignore_index=True)

        df_profit_by_date_historico = pd.read_csv('profit_by_date_historico_5ano_LSTM_default.csv')  
        df_profit_by_date_historico = pd.concat([df_profit_by_date_historico, df_profit_by_date], ignore_index=True)

        df_operacoes_historico = pd.read_csv('operacoes_dia_historico_5ano_LSTM_default.csv')  
        df_operacoes_historico = pd.concat([df_operacoes_historico, df_operacoes], ignore_index=True)

        df_negociacoes_valores_historico.to_csv('negociações_valores_historico_5ano_LSTM_default.csv', index=False)
        df_profit_by_date_historico.to_csv('profit_by_date_historico_5ano_LSTM_default.csv', index=False)
        df_operacoes_historico.to_csv('operacoes_dia_historico_5ano_LSTM_default.csv', index=False)

        arquivo = "models/dqn.h5"
        inicio = str(inicio_dados_teste)
        fim = str(fim_dados_teste)
        nome_novo = "models/testados_janelas_moveis/dqn_"+inicio+"_"+fim +"_LSTM_default.h5"

        if os.path.exists(arquivo):
            # Renomeia o arquivo
            os.rename(arquivo, nome_novo)




### CNN

In [ ]:
#Importar 
import os
import pandas as pd
import logging
import coloredlogs
from docopt import docopt
from trading_bot.agent_CNN import Agent
from trading_bot.methods_default import train_model, evaluate_model
from trading_bot.utils import (
    get_stock_data,
    get_data_with_date,
    format_currency,
    format_position,
    show_train_result,
    switch_k_backend_device
)

#Dados
dados = "data/ibovespa_2000_2024.csv"

#Import como lista
dados_lista = get_stock_data(dados)
dados_lista_data = get_data_with_date(dados)

#Valores janela movel
janela_inicial = 1260  
delta_janela = 1260
final_dados = len(dados_lista)
cont = 0
inicio_janela = 0   
final_janela = janela_inicial   
 
#Eps treino
ep_count=int(3)

while cont < final_dados:
    print(inicio_janela, final_janela)
    dados_treino = dados_lista[inicio_janela:final_janela]
    agent = Agent(int(10), strategy='dqn', pretrained=False, model_name=None)
    ep_count=int(3)
    episode = None
    for episode in range(1, ep_count + 1):
        treino = train_model(agent, episode, dados_treino, ep_count=ep_count,
                                   batch_size=int(32), ws=int(10))

    # Se tem dados para o próximo teste   
    if final_janela + delta_janela + 10 < final_dados:
        inicio_dados_teste = final_janela
        fim_dados_teste = inicio_dados_teste + delta_janela + 1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        inicio_janela += delta_janela
        final_janela += delta_janela

    else:
        inicio_dados_teste = final_janela
        fim_dados_teste = final_dados+1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        cont = final_dados

    
    if len(dados_teste) > 10: 
        #Teste
        initial_offset = dados_teste[1] - dados_teste[0]
        agent = Agent(int(10), strategy='dqn', pretrained=True, model_name='dqn.h5')
        profit, _ = evaluate_model(agent, dados_teste, dados_teste_data, ws=int(10), debug=True)

        # Salvar resultados teste
        df_negociacoes_valores = pd.read_csv('negociações_valores.csv')
        df_profit_by_date = pd.read_csv('profit_by_date.csv')
        df_operacoes = pd.read_csv('operação_dia.csv')

        df_negociacoes_valores_historico = pd.read_csv('negociações_valores_historico_5ano_CNN_default.csv') 
        df_negociacoes_valores_historico = pd.concat([df_negociacoes_valores_historico, df_negociacoes_valores], ignore_index=True)

        df_profit_by_date_historico = pd.read_csv('profit_by_date_historico_5ano_CNN_default.csv')  
        df_profit_by_date_historico = pd.concat([df_profit_by_date_historico, df_profit_by_date], ignore_index=True)

        df_operacoes_historico = pd.read_csv('operacoes_dia_historico_5ano_CNN_default.csv')  
        df_operacoes_historico = pd.concat([df_operacoes_historico, df_operacoes], ignore_index=True)

        df_negociacoes_valores_historico.to_csv('negociações_valores_historico_5ano_CNN_default.csv', index=False)
        df_profit_by_date_historico.to_csv('profit_by_date_historico_5ano_CNN_default.csv', index=False)
        df_operacoes_historico.to_csv('operacoes_dia_historico_5ano_CNN_default.csv', index=False)

        arquivo = "models/dqn.h5"
        inicio = str(inicio_dados_teste)
        fim = str(fim_dados_teste)
        nome_novo = "models/testados_janelas_moveis/dqn_"+inicio+"_"+fim +"_CNN_default.h5"

        if os.path.exists(arquivo):
            # Renomeia o arquivo
            os.rename(arquivo, nome_novo)


## Modelos que utilizam a função recompensa "passo"

### Dense

In [ ]:
#Importar 
import os
import pandas as pd
import logging
import coloredlogs
from docopt import docopt
from trading_bot.agent_Dense import Agent
from trading_bot.methods_step import train_model, evaluate_model
from trading_bot.utils import (
    get_stock_data,
    get_data_with_date,
    format_currency,
    format_position,
    show_train_result,
    switch_k_backend_device
)

#Dados
dados = "data/ibovespa_2000_2024.csv"

#Import como lista
dados_lista = get_stock_data(dados)
dados_lista_data = get_data_with_date(dados)

#Valores janela movel
janela_inicial = 1260  
delta_janela = 1260
final_dados = len(dados_lista)
cont = 0
inicio_janela = 0   
final_janela = janela_inicial  
 
#Eps treino
ep_count=int(3)

while cont < final_dados:
    print(inicio_janela, final_janela)
    dados_treino = dados_lista[inicio_janela:final_janela]
    agent = Agent(int(10), strategy='dqn', pretrained=False, model_name=None)
    ep_count=int(3)
    episode = None
    for episode in range(1, ep_count + 1):
        treino = train_model(agent, episode, dados_treino, ep_count=ep_count,
                                   batch_size=int(32), ws=int(10))

    # Se tem dados para o próximo teste   
    if final_janela + delta_janela + 10 < final_dados:
        inicio_dados_teste = final_janela
        fim_dados_teste = inicio_dados_teste + delta_janela + 1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        inicio_janela += delta_janela
        final_janela += delta_janela

    else:
        inicio_dados_teste = final_janela
        fim_dados_teste = final_dados+1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        cont = final_dados
    
    if len(dados_teste) > 10: 
        #Teste
        initial_offset = dados_teste[1] - dados_teste[0]
        agent = Agent(int(10), strategy='dqn', pretrained=True, model_name='dqn.h5')
        profit, _ = evaluate_model(agent, dados_teste, dados_teste_data, ws=int(10), debug=True)

        # Salvar resultados teste
        df_negociacoes_valores = pd.read_csv('negociações_valores.csv')
        df_profit_by_date = pd.read_csv('profit_by_date.csv')
        df_operacoes = pd.read_csv('operação_dia.csv')

        df_negociacoes_valores_historico = pd.read_csv('negociações_valores_historico_5ano_Dense_step.csv') 
        df_negociacoes_valores_historico = pd.concat([df_negociacoes_valores_historico, df_negociacoes_valores], ignore_index=True)

        df_profit_by_date_historico = pd.read_csv('profit_by_date_historico_5ano_Dense_step.csv') 
        df_profit_by_date_historico = pd.concat([df_profit_by_date_historico, df_profit_by_date], ignore_index=True)

        df_operacoes_historico = pd.read_csv('operacoes_dia_historico_5ano_Dense_step.csv') 
        df_operacoes_historico = pd.concat([df_operacoes_historico, df_operacoes], ignore_index=True)

        df_negociacoes_valores_historico.to_csv('negociações_valores_historico_5ano_Dense_step.csv', index=False)
        df_profit_by_date_historico.to_csv('profit_by_date_historico_5ano_Dense_step.csv', index=False)
        df_operacoes_historico.to_csv('operacoes_dia_historico_5ano_Dense_step.csv', index=False)

        arquivo = "models/dqn.h5"
        inicio = str(inicio_dados_teste)
        fim = str(fim_dados_teste)
        nome_novo = "models/testados_janelas_moveis/dqn_"+inicio+"_"+fim +"_Dense_step.h5"

        if os.path.exists(arquivo):
            # Renomeia o arquivo
            os.rename(arquivo, nome_novo)


### LSTM

In [ ]:
#Importar 
import os
import pandas as pd
import logging
import coloredlogs
from docopt import docopt
from trading_bot.agent_LSTM import Agent
from trading_bot.methods_step import train_model, evaluate_model
from trading_bot.utils import (
    get_stock_data,
    get_data_with_date,
    format_currency,
    format_position,
    show_train_result,
    switch_k_backend_device
)

#Dados
dados = "data/ibovespa_2000_2024.csv"

#Import como lista
dados_lista = get_stock_data(dados)
dados_lista_data = get_data_with_date(dados)

#Valores janela movel
janela_inicial = 1260  
delta_janela = 1260
final_dados = len(dados_lista)
cont = 0
inicio_janela = 0  
final_janela = janela_inicial   
 
#Eps treino
ep_count=int(3)

while cont < final_dados:
    print(inicio_janela, final_janela)
    dados_treino = dados_lista[inicio_janela:final_janela]
    agent = Agent(int(10), strategy='dqn', pretrained=False, model_name=None)
    ep_count=int(3)
    episode = None
    for episode in range(1, ep_count + 1):
        treino = train_model(agent, episode, dados_treino, ep_count=ep_count,
                                   batch_size=int(32), ws=int(10))

    # Se tem dados para o próximo teste   
    if final_janela + delta_janela + 10 < final_dados:
        inicio_dados_teste = final_janela
        fim_dados_teste = inicio_dados_teste + delta_janela + 1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        inicio_janela += delta_janela
        final_janela += delta_janela

    else:
        inicio_dados_teste = final_janela
        fim_dados_teste = final_dados+1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        cont = final_dados
    
    if len(dados_teste) > 10: 
        #Teste
        initial_offset = dados_teste[1] - dados_teste[0]
        agent = Agent(int(10), strategy='dqn', pretrained=True, model_name='dqn.h5')
        profit, _ = evaluate_model(agent, dados_teste, dados_teste_data, ws=int(10), debug=True)

        # Salvar resultados teste
        df_negociacoes_valores = pd.read_csv('negociações_valores.csv')
        df_profit_by_date = pd.read_csv('profit_by_date.csv')
        df_operacoes = pd.read_csv('operação_dia.csv')

        df_negociacoes_valores_historico = pd.read_csv('negociações_valores_historico_5ano_LSTM_step.csv')
        df_negociacoes_valores_historico = pd.concat([df_negociacoes_valores_historico, df_negociacoes_valores], ignore_index=True)

        df_profit_by_date_historico = pd.read_csv('profit_by_date_historico_5ano_LSTM_step.csv')  
        df_profit_by_date_historico = pd.concat([df_profit_by_date_historico, df_profit_by_date], ignore_index=True)

        df_operacoes_historico = pd.read_csv('operacoes_dia_historico_5ano_LSTM_step.csv')  
        df_operacoes_historico = pd.concat([df_operacoes_historico, df_operacoes], ignore_index=True)

        df_negociacoes_valores_historico.to_csv('negociações_valores_historico_5ano_LSTM_step.csv', index=False)
        df_profit_by_date_historico.to_csv('profit_by_date_historico_5ano_LSTM_step.csv', index=False)
        df_operacoes_historico.to_csv('operacoes_dia_historico_5ano_LSTM_step.csv', index=False)

        arquivo = "models/dqn.h5"
        inicio = str(inicio_dados_teste)
        fim = str(fim_dados_teste)
        nome_novo = "models/testados_janelas_moveis/dqn_"+inicio+"_"+fim +"_LSTM_step.h5"

        if os.path.exists(arquivo):
            # Renomeia o arquivo
            os.rename(arquivo, nome_novo)


### CNN

In [ ]:
#Importar 
import os
import pandas as pd
import logging
import coloredlogs
from docopt import docopt
from trading_bot.agent_CNN import Agent
from trading_bot.methods_step import train_model, evaluate_model
from trading_bot.utils import (
    get_stock_data,
    get_data_with_date,
    format_currency,
    format_position,
    show_train_result,
    switch_k_backend_device
)

#Dados
dados = "data/ibovespa_2000_2024.csv"

#Import como lista
dados_lista = get_stock_data(dados)
dados_lista_data = get_data_with_date(dados)

#Valores janela movel
janela_inicial = 1260  
delta_janela = 1260
final_dados = len(dados_lista)
cont = 0
inicio_janela = 0   
final_janela = janela_inicial   
 
#Eps treino
ep_count=int(3)

while cont < final_dados:
    print(inicio_janela, final_janela)
    dados_treino = dados_lista[inicio_janela:final_janela]
    agent = Agent(int(10), strategy='dqn', pretrained=False, model_name=None)
    ep_count=int(3)
    episode = None
    for episode in range(1, ep_count + 1):
        treino = train_model(agent, episode, dados_treino, ep_count=ep_count,
                                   batch_size=int(32), ws=int(10))

    # Se tem dados para o próximo teste   
    if final_janela + delta_janela + 10 < final_dados:
        inicio_dados_teste = final_janela
        fim_dados_teste = inicio_dados_teste + delta_janela + 1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        inicio_janela += delta_janela
        final_janela += delta_janela

    else:
        inicio_dados_teste = final_janela
        fim_dados_teste = final_dados+1
        dados_teste = dados_lista[inicio_dados_teste:fim_dados_teste]
        dados_teste_data = dados_lista_data[inicio_dados_teste:fim_dados_teste]
        cont = final_dados
    
    if len(dados_teste) > 10: 
        #Teste
        initial_offset = dados_teste[1] - dados_teste[0]
        agent = Agent(int(10), strategy='dqn', pretrained=True, model_name='dqn.h5')
        profit, _ = evaluate_model(agent, dados_teste, dados_teste_data, ws=int(10), debug=True)

        # Salvar resultados teste
        df_negociacoes_valores = pd.read_csv('negociações_valores.csv')
        df_profit_by_date = pd.read_csv('profit_by_date.csv')
        df_operacoes = pd.read_csv('operação_dia.csv')

        df_negociacoes_valores_historico = pd.read_csv('negociações_valores_historico_5ano_CNN_step.csv') 
        df_negociacoes_valores_historico = pd.concat([df_negociacoes_valores_historico, df_negociacoes_valores], ignore_index=True)

        df_profit_by_date_historico = pd.read_csv('profit_by_date_historico_5ano_CNN_step.csv')  
        df_profit_by_date_historico = pd.concat([df_profit_by_date_historico, df_profit_by_date], ignore_index=True)

        df_operacoes_historico = pd.read_csv('operacoes_dia_historico_5ano_CNN_step.csv')  
        df_operacoes_historico = pd.concat([df_operacoes_historico, df_operacoes], ignore_index=True)

        df_negociacoes_valores_historico.to_csv('negociações_valores_historico_5ano_CNN_step.csv', index=False)
        df_profit_by_date_historico.to_csv('profit_by_date_historico_5ano_CNN_step.csv', index=False)
        df_operacoes_historico.to_csv('operacoes_dia_historico_5ano_CNN_step.csv', index=False)

        arquivo = "models/dqn.h5"
        inicio = str(inicio_dados_teste)
        fim = str(fim_dados_teste)
        nome_novo = "models/testados_janelas_moveis/dqn_"+inicio+"_"+fim +"_CNN_step.h5"

        if os.path.exists(arquivo):
            # Renomeia o arquivo
            os.rename(arquivo, nome_novo)
